In [1]:
import os
import sys
import pandas as pd
import numpy as np
import geocoder
import feather
import re
from tqdm import tqdm_notebook as tqdm

sys.path.append("/Users/ueda/Desktop/kaggle/mynavi/code/src/")
from util_tool import reduce_mem_usage

In [2]:
train = pd.read_csv('../input/train.csv')
target = train["賃料"]
#train = train.drop("賃料", axis=1)
test = pd.read_csv('../input/test.csv')

train_length = train.shape[0]
test_length = test.shape[0]
all_df = pd.concat([train, test], axis=0, ignore_index=True)


access = feather.read_dataframe("../code/feature_csv/access_feature.feather")
all_df = pd.concat([all_df, access], axis=1)
base = feather.read_dataframe("../code/feature_csv/base_feature1.feather")
all_df = pd.concat([all_df, base], axis=1)

all_df = reduce_mem_usage(all_df)
org_columns = all_df.columns

/Users/ueda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


Memory usage of dataframe is 80.05 MB
column =  631
0
50
100
150
200
250
300
350
400
450
500
550
600
Memory usage after optimization is: 64.31 MB
Decreased by 19.7%


In [3]:
def modify_miss(df_input):
    df_input.loc[df_input["間取り"].str.contains("11R"), "間取り"] = "1R"
    
    df_input.loc[df_input["築年数"].str.contains("520年5ヶ月"), "築年数"] = "52年5ヶ月"
    
    df_input.loc[df_input["築年数"].str.contains("1019年7ヶ月"), "築年数"] = "19年7ヶ月"
    
    df_input.loc[df_input["面積"].str.contains("430.1m2"), "面積"] = "43.01m2"
    #idx = df_input["築年数"].str.contains("1019年7ヶ月")
    #df_input["築年数"][idx] = "19年7ヶ月"    
    
    "三田線 三田(兵庫県)駅 徒歩14分"
    
    return df_input

all_df = modify_miss(all_df)

In [4]:
org_columns.values

array(['id', 'アクセス', 'キッチン', 'バス・トイレ', '周辺環境', '契約期間', '室内設備', '建物構造',
       '所在地', '所在階', '放送・通信', '方角', '築年数', '賃料', '間取り', '面積', '駐車場',
       '路線_0', '駅_0', '駅からの距離_0', '路線_1', '駅_1', '駅からの距離_1', '路線_2',
       '駅_2', '駅からの距離_2', '駅_0からの距離_mean', '駅_0からの距離_std', '駅_0からの距離_min',
       '駅_0からの距離_max', '駅_1からの距離_mean', '駅_1からの距離_std', '駅_1からの距離_min',
       '駅_1からの距離_max', '駅_2からの距離_mean', '駅_2からの距離_std', '駅_2からの距離_min',
       '駅_2からの距離_max', '路線_0からの距離_mean', '路線_0からの距離_std', '路線_0からの距離_min',
       '路線_0からの距離_max', '路線_1からの距離_mean', '路線_1からの距離_std',
       '路線_1からの距離_min', '路線_1からの距離_max', '路線_2からの距離_mean',
       '路線_2からの距離_std', '路線_2からの距離_min', '路線_2からの距離_max', '路線_0_counts',
       '路線_1_counts', '路線_2_counts', '駅_0_counts', '駅_1_counts',
       '駅_2_counts', '路線_counts_max', '路線_counts_mean', '路線_counts_min',
       '路線_counts_std', '駅_counts_max', '駅_counts_mean', '駅_counts_min',
       '駅_counts_std', '距離_max', '距離_min', '距離_mean', '距離_std',
       '路線_0_counts/距離', '路線_1_

In [5]:
near_cols = ['near_池袋駅', 'near_綾瀬駅', 'near_三軒茶屋駅',
       'near_錦糸町駅', 'near_葛西駅', 'near_蒲田駅', 'near_小岩駅', 'near_門前仲町駅',
       'near_月島駅', 'near_中野(東京都)駅', 'near_西新井駅', 'near_北千住駅', 'near_新小岩駅',
       'near_高円寺駅', 'near_住吉(東京都)駅', 'near_荻窪駅', 'near_大森(東京都)駅',
       'near_押上駅', 'near_西葛西駅', 'near_北綾瀬駅', 'near_笹塚駅', 'near_亀有駅',
       'near_大泉学園駅', 'near_麻布十番駅', 'near_阿佐ケ谷駅', 'near_千歳烏山駅',
       'near_恵比寿駅', 'near_学芸大学駅', 'near_大塚(東京都)駅', 'near_青井駅',
       'near_駒沢大学駅', 'near_越中島駅', 'near_新高円寺駅', 'near_浅草駅',
       'near_板橋区役所前駅', 'near_板橋駅', 'near_広尾駅', 'near_木場(東京都)駅',
       'near_目黒駅', 'near_瑞江駅', 'near_篠崎駅', 'near_芦花公園駅', 'near_京急蒲田駅',
       'near_浅草橋駅', 'near_三ノ輪駅', 'near_板橋本町駅', 'near_一之江駅', 'near_白金高輪駅',
       'near_五反野駅', 'near_菊川(東京都)駅', 'near_勝どき駅', 'near_武蔵小山駅',
       'near_梅島駅', 'near_新中野駅', 'near_石神井公園駅', 'near_東高円寺駅', 'near_練馬駅',
       'near_都立大学駅', 'near_大井町駅', 'near_赤羽駅', 'near_王子駅', 'near_自由が丘駅',
       'near_巣鴨駅', 'near_八丁堀(東京都)駅', 'near_船堀駅', 'near_お花茶屋駅', 'near_築地駅',
       'near_要町駅', 'near_池尻大橋駅', 'near_代田橋駅', 'near_東十条駅', 'near_中野坂上駅',
       'near_神楽坂駅', 'near_大山(東京都)駅', 'near_下板橋駅', 'near_桜新町駅',
       'near_清澄白河駅', 'near_西大島駅', 'near_入谷(東京都)駅', 'near_八幡山駅',
       'near_中板橋駅', 'near_江戸川橋駅', 'near_金町(東京都)駅', 'near_祖師ケ谷大蔵駅',
       'near_南千住駅', 'near_千歳船橋駅', 'near_大師前駅', 'near_駒込駅', 'near_南阿佐ケ谷駅',
       'near_方南町駅', 'near_新江古田駅', 'near_祐天寺駅', 'near_西荻窪駅', 'near_六町駅',
       'near_中目黒駅', 'near_森下(東京都)駅', 'near_本蓮沼駅', 'near_王子神谷駅',
       'near_豊洲駅', 'near_田町(東京都)駅', 'near_亀戸駅', 'near_鷺ノ宮駅', 'near_竹ノ塚駅',
       'near_馬込駅', 'near_京成立石駅', 'near_西新宿五丁目駅', 'near_池上駅', 'near_六本木駅',
       'near_五反田駅', 'near_白金台駅', 'near_新大塚駅', 'near_地下鉄赤塚駅', 'near_西馬込駅',
       'near_中野新橋駅', 'near_新富町(東京都)駅', 'near_用賀駅', 'near_成増駅',
       'near_大森海岸駅', 'near_東陽町駅', 'near_京成小岩駅', 'near_西太子堂駅', 'near_青砥駅',
       'near_落合南長崎駅', 'near_下北沢駅', 'near_北池袋駅', 'near_三田(東京都)駅',
       'near_高田馬場駅', 'near_幡ケ谷駅', 'near_練馬高野台駅', 'near_成城学園前駅',
       'near_沼袋駅', 'near_東武練馬駅', 'near_人形町駅', 'near_不動前駅', 'near_江古田駅',
       'near_東中野駅', 'near_椎名町駅', 'near_梅屋敷(東京都)駅', 'near_初台駅',
       'near_西日暮里駅', 'near_大岡山駅', 'near_水天宮前駅', 'near_経堂駅', 'near_平和島駅',
       'near_赤羽橋駅', 'near_京成金町駅', 'near_品川駅', 'near_早稲田(東京メトロ)駅',
       'near_渋谷駅', 'near_明大前駅', 'near_十条(東京都)駅', 'near_西小山駅',
       'near_中野富士見町駅', 'near_馬喰町駅', 'near_千石駅', 'near_中村橋駅', 'near_上板橋駅',
       'near_千住大橋駅', 'near_地下鉄成増駅', 'near_東長崎駅', 'near_西巣鴨駅',
       'near_都立家政駅', 'near_蔵前駅', 'near_飯田橋駅', 'near_大島(東京都)駅',
       'near_秋葉原駅', 'near_田端駅', 'near_蓮沼駅', 'near_平和台(東京都)駅', 'near_下井草駅',
       'near_富士見台駅', 'near_両国(都営線)駅', 'near_下赤塚駅', 'near_ときわ台(東京都)駅',
       'near_桜上水駅', 'near_本所吾妻橋駅', 'near_大森町駅', 'near_志村坂上駅', 'near_野方駅',
       'near_代官山駅', 'near_目白駅', 'near_浅草(ＴＸ)駅', 'near_表参道駅', 'near_東池袋駅',
       'near_茅場町駅', 'near_日暮里駅', 'near_新井薬師前駅', 'near_堀切菖蒲園駅',
       'near_上石神井駅', 'near_下高井戸駅', 'near_桜台(東京都)駅', 'near_新板橋駅',
       'near_二子玉川駅', 'near_新御徒町駅', 'near_武蔵関駅', 'near_春日(東京都)駅',
       'near_泉岳寺駅', 'near_上北沢駅', 'near_鶯谷駅', 'near_戸越銀座駅', 'near_井荻駅',
       'near_平井(東京都)駅', 'near_西大井駅', 'near_高井戸駅', 'near_若林(東京都)駅',
       'near_富士見ケ丘駅', 'near_志村三丁目駅', 'near_柴又駅', 'near_小竹向原駅',
       'near_久我山駅', 'near_葛西臨海公園駅', 'near_後楽園駅', 'near_江戸川駅', 'near_曳舟駅',
       'near_両国駅', 'near_代々木上原駅', 'near_西新井大師西駅', 'near_保谷駅', 'near_矢口渡駅',
       'near_糀谷駅', 'near_京成曳舟駅', 'near_千鳥町駅', 'near_町屋(東京メトロ)駅',
       'near_新桜台駅', 'near_赤羽岩淵駅', 'near_武蔵新田駅', 'near_中延駅', 'near_京成高砂駅',
       'near_落合(東京都)駅', 'near_小菅駅', 'near_千川駅', 'near_上野毛駅', 'near_大崎駅',
       'near_上野駅', 'near_上井草駅', 'near_乃木坂駅', 'near_上町駅', 'near_白山(東京都)駅',
       'near_蓮根駅', 'near_本駒込駅', 'near_洗足駅', 'near_仙川駅', 'near_氷川台駅',
       'near_三河島駅', 'near_光が丘駅', 'near_西台駅', 'near_雑司が谷駅', 'near_浜松町駅',
       'near_御茶ノ水駅', 'near_戸越駅', 'near_志茂駅', 'near_緑が丘(東京都)駅',
       'near_大久保(東京都)駅', 'near_中井駅', 'near_牛込神楽坂駅', 'near_雑色駅',
       'near_茗荷谷駅', 'near_新宿駅', 'near_浜町駅', 'near_とうきょうスカイツリー駅',
       'near_旗の台駅', 'near_荏原町駅', 'near_牛込柳町駅', 'near_等々力駅', 'near_吉祥寺駅',
       'near_稲荷町(東京都)駅', 'near_立会川駅', 'near_長原(東京都)駅', 'near_四ツ木駅',
       'near_松陰神社前駅', 'near_永福町駅', 'near_本郷三丁目駅', 'near_江北駅',
       'near_駒場東大前駅', 'near_東大島駅', 'near_下落合駅', 'near_谷在家駅', 'near_舎人公園駅',
       'near_練馬春日町駅', 'near_西新宿駅', 'near_東向島駅', 'near_田原町(東京都)駅',
       'near_芝公園駅', 'near_奥沢駅', 'near_新大久保駅', 'near_南砂町駅', 'near_上中里駅',
       'near_浜田山駅', 'near_西永福駅', 'near_馬喰横山駅', 'near_池ノ上駅', 'near_西ケ原駅',
       'near_赤坂(東京都)駅', 'near_牛田(東京都)駅', 'near_大鳥居駅', 'near_東大前駅',
       'near_神田(東京都)駅', 'near_雪が谷大塚駅', 'near_松原(東京都)駅', 'near_大門(東京都)駅',
       'near_扇大橋駅', 'near_尾山台駅', 'near_北赤羽駅', 'near_岩本町駅', 'near_高輪台駅',
       'near_若松河田駅', 'near_潮見駅', 'near_東北沢駅', 'near_護国寺駅', 'near_東日本橋駅',
       'near_世田谷駅', 'near_亀戸水神駅', 'near_下神明駅', 'near_千駄木駅', 'near_曙橋駅',
       'near_豪徳寺駅', 'near_神泉駅', 'near_東新宿駅', 'near_尾久駅', 'near_梅ケ丘駅',
       'near_御徒町駅', 'near_御成門駅', 'near_西早稲田駅', 'near_浦安(千葉県)駅',
       'near_宝町(東京都)駅', 'near_世田谷代田駅', 'near_舎人駅', 'near_天王洲アイル駅',
       'near_高野(東京都)駅', 'near_北品川駅', 'near_高島平駅', 'near_三ノ輪橋駅',
       'near_東松原駅', 'near_石川台駅', 'near_六本木一丁目駅', 'near_神谷町駅', 'near_洗足池駅',
       'near_青物横丁駅', 'near_戸越公園駅', 'near_新馬場駅', 'near_水道橋駅', 'near_参宮橋駅',
       'near_新柴又駅', 'near_荏原中延駅', 'near_市ケ谷駅', 'near_浮間舟渡駅', 'near_新代田駅',
       'near_三鷹台駅', 'near_新三河島駅', 'near_小伝馬町駅', 'near_御嶽山駅',
       'near_豊島園(都営線)駅', 'near_九品仏駅', 'near_東あずま駅', 'near_八広駅',
       'near_代々木八幡駅', 'near_北千束駅', 'near_湯島駅', 'near_新橋駅', 'near_代々木公園駅',
       'near_芝浦ふ頭駅', 'near_下丸子駅', 'near_四谷三丁目駅', 'near_宮の坂駅',
       'near_豊島園(西武線)駅', 'near_鮫洲駅', 'near_久が原駅', 'near_六郷土手駅',
       'near_多摩川駅', 'near_汐留駅', 'near_品川シーサイド駅', 'near_京成関屋駅',
       'near_東雲(東京都)駅', 'near_田園調布駅', 'near_喜多見駅', 'near_小村井駅',
       'near_大崎広小路駅', 'near_新宿三丁目駅', 'near_神保町駅', 'near_末広町(東京都)駅',
       'near_東銀座駅', 'near_沼部駅', 'near_辰巳駅', 'near_代々木駅', 'near_新宿御苑前駅',
       'near_西高島平駅', 'near_淡路町駅', 'near_新御茶ノ水駅', 'near_九段下駅',
       'near_見沼代親水公園駅', 'near_都庁前駅', 'near_四ツ谷駅', 'near_赤土小学校前駅',
       'near_熊野前(舎人ライナー)駅', 'near_鐘ケ淵駅', 'near_東伏見駅', 'near_仲御徒町駅',
       'near_溜池山王駅', 'near_根津駅', 'near_新高島平駅', 'near_穴守稲荷駅',
       'near_青山一丁目駅', 'near_ひばりケ丘(東京都)駅', 'near_赤坂見附駅', 'near_新豊洲駅',
       'near_麹町駅', 'near_足立小台駅', 'near_大塚駅前駅', 'near_東京駅', 'near_半蔵門駅',
       'near_和光市駅', 'near_西武新宿駅', 'near_外苑前駅', 'near_大井競馬場前駅', 'near_八潮駅',
       'near_堀切駅', 'near_日本橋(東京都)駅', 'near_谷塚駅', 'near_原宿駅',
       'near_小川町(東京都)駅', 'near_鵜の木駅', 'near_南新宿駅', 'near_築地市場駅',
       'near_明治神宮前駅', 'near_山下(東京都)駅', 'near_銀座駅', 'near_虎ノ門駅',
       'near_永田町駅', 'near_北参道駅', 'near_三越前駅', 'near_巣鴨新田駅', 'near_千駄ケ谷駅',
       'near_町屋(京成線)駅', 'near_向原(東京都)駅', 'near_早稲田(都電荒川線)駅', 'near_熊野前駅',
       'near_信濃町駅', 'near_上野広小路駅', 'near_狛江駅', 'near_王子駅前駅', 'near_町屋駅前駅',
       'near_昭和島駅', 'near_日の出(東京都)駅', 'near_京橋(東京都)駅', 'near_宮ノ前駅',
       'near_つつじケ丘駅', 'near_荒川一中前駅', 'near_梶原駅', 'near_東尾久三丁目駅',
       'near_有明テニスの森駅', 'near_新日本橋駅', 'near_栄町(東京都)駅', 'near_面影橋駅',
       'near_庚申塚駅', 'near_新庚申塚駅', 'near_大手町(東京都)駅', 'near_銀座一丁目駅',
       'near_井の頭公園駅', 'near_天空橋駅', 'near_南行徳駅', 'near_竹橋駅', 'near_学習院下駅',
       'near_荒川七丁目駅', 'near_朝霞駅', 'near_町屋二丁目駅', 'near_妙典駅',
       'near_国立競技場駅', 'near_西ケ原四丁目駅', 'near_小台駅', 'near_国際展示場駅',
       'near_整備場駅', 'near_新木場駅', 'near_お台場海浜公園駅', 'near_内幸町駅',
       'near_荒川区役所前駅', 'near_二子新地駅', 'near_市場前駅', 'near_滝野川一丁目駅',
       'near_荒川遊園地前駅', 'near_有明(東京都)駅', 'near_新宿西口駅', 'near_港町駅',
       'near_飛鳥山駅', 'near_荒川車庫前駅', 'near_日暮里(舎人ライナー)駅', 'near_東池袋四丁目駅',
       'near_鬼子母神前駅', 'near_市川駅']

In [6]:
%%time
targets = ['float面積', "築月数", 'contract_period']    #所在_区 での面積　あたりの値段
columns = near_cols
stats = ["mean", "std", "min", "max"]

new_cols = []
for target in tqdm(targets):
    new_df = all_df[targets]
    for column in tqdm(columns):
        if target != column:
            for stat in stats:
                temp = all_df.groupby(column)[target].agg([stat]).rename({stat:f'{target}{stat}_of_{column}'},axis=1)
                temp[f'{target}{stat}_of_{column}'][0] = np.nan
                #all_df[f'{target}{stat}_of_{column}'] = all_df[column].map(temp.to_dict()[f'{target}{stat}_of_{column}'])
                
                new_df[f'{target}{stat}_of_{column}'] = all_df[column].map(temp.to_dict()[f'{target}{stat}_of_{column}'])
                new_cols.append(new_df[f'{target}{stat}_of_{column}'])
                
                #all_df = pd.merge(all_df,temp,on=column,how='left')
            if target != "賃料":
                #all_df[f'平均{target}_diff_{column}'] = all_df[f'{target}mean_of_{column}'] - all_df[target]
                #all_df[f'平均{target}_percent_{column}'] = all_df[f'{target}mean_of_{column}'] / all_df[target]

                new_df[f'平均{target}_diff_{column}'] = new_df[f'{target}mean_of_{column}'] - new_df[target]
                new_df[f'平均{target}_percent_{column}'] = new_df[f'{target}mean_of_{column}'] / new_df[target]  
                new_cols.append(new_df[f'平均{target}_diff_{column}'])
                new_cols.append(new_df[f'平均{target}_percent_{column}'])
                
all_df_feature = pd.concat(new_cols, axis=1)

/Users/ueda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/Users/ueda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/ueda/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co



CPU times: user 1min 17s, sys: 49.5 s, total: 2min 7s
Wall time: 2min 14s


In [7]:
"""index ずれていないかチェック"""
#all_df_feature[all_df["near_市川駅"] == 1]

'index ずれていないかチェック'

In [8]:
"""column = "アクセス"
targets = ['int面積','賃料']
for target in targets:
    for stat in stats:
        temp = all_df.groupby(column)[target].agg([stat]).rename({stat:f'{target}{stat}_of_{column}'},axis=1)
        all_df = pd.merge(all_df,temp,on=column,how='left')"""
 

'column = "アクセス"\ntargets = [\'int面積\',\'賃料\']\nfor target in targets:\n    for stat in stats:\n        temp = all_df.groupby(column)[target].agg([stat]).rename({stat:f\'{target}{stat}_of_{column}\'},axis=1)\n        all_df = pd.merge(all_df,temp,on=column,how=\'left\')'

In [9]:
#all_df_feature.describe()

In [10]:
#new_columns = [column for column in all_df.columns if column not in org_columns]

In [16]:
length = len(list(all_df_feature.columns))
all_colList = list(all_df_feature.columns)
t = []
memreduced_all_df_feature = pd.DataFrame()
for i in tqdm(range(20)):
    cols = all_colList[int(i*length/20):int((i+1)*length/20)]
    memreduced_all_df_feature[cols] = reduce_mem_usage(all_df_feature[cols])

Memory usage of dataframe is 170.62 MB
column =  439
0
50
100
150


/Users/ueda/Desktop/kaggle/mynavi/code/src/util_tool.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(np.float32)


200
250
300
350
400
Memory usage after optimization is: 105.05 MB
Decreased by 38.4%
Memory usage of dataframe is 210.11 MB
column =  439
0
50
100
150
200
250
300
350
400
Memory usage after optimization is: 105.05 MB
Decreased by 50.0%
Memory usage of dataframe is 210.11 MB
column =  439
0
50
100
150
200
250
300
350
400
Memory usage after optimization is: 105.05 MB
Decreased by 50.0%
Memory usage of dataframe is 210.11 MB
column =  439
0
50
100
150
200
250
300
350
400
Memory usage after optimization is: 105.05 MB
Decreased by 50.0%
Memory usage of dataframe is 210.59 MB
column =  440
0
50
100
150
200
250
300
350
400
Memory usage after optimization is: 105.29 MB
Decreased by 50.0%
Memory usage of dataframe is 210.11 MB
column =  439
0
50
100
150
200
250
300
350
400
Memory usage after optimization is: 105.05 MB
Decreased by 50.0%
Memory usage of dataframe is 210.11 MB
column =  439
0
50
100
150
200
250


/Users/ueda/Desktop/kaggle/mynavi/code/src/util_tool.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(np.float32)


300
350
400
Memory usage after optimization is: 105.05 MB
Decreased by 50.0%
Memory usage of dataframe is 210.11 MB
column =  439
0
50
100
150
200
250
300
350
400
Memory usage after optimization is: 105.05 MB
Decreased by 50.0%
Memory usage of dataframe is 210.11 MB
column =  439
0
50
100
150
200
250
300
350
400
Memory usage after optimization is: 105.05 MB
Decreased by 50.0%
Memory usage of dataframe is 210.59 MB
column =  440
0
50
100
150
200
250
300
350
400
Memory usage after optimization is: 105.29 MB
Decreased by 50.0%
Memory usage of dataframe is 210.11 MB
column =  439
0
50
100
150
200
250
300
350
400
Memory usage after optimization is: 105.05 MB
Decreased by 50.0%
Memory usage of dataframe is 210.11 MB
column =  439
0
50
100
150
200
250
300
350
400
Memory usage after optimization is: 105.05 MB
Decreased by 50.0%
Memory usage of dataframe is 210.11 MB
column =  439
0
50
100
150
200
250
300
350
400
Memory usage after optimization is: 105.05 MB
Decreased by 50.0%
Memory usage of d

In [21]:
"""カラムがずれていないか　check """
list(all_df_feature.columns) == list(memreduced_all_df_feature.columns)

True

In [22]:
#all_df_feature = all_df_feature.drop(['緯度経度', "count_id"], axis=1)
output_dir = '../code/feature_csv/'
memreduced_all_df_feature.to_feather(os.path.join(output_dir, 'nonleak_nearStation_group_feature.feather'))

In [23]:
memreduced_all_df_feature.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62732 entries, 0 to 62731
Columns: 8784 entries, float面積mean_of_near_池袋駅 to 平均contract_period_percent_near_市川駅
dtypes: float32(8784)
memory usage: 2.1 GB


In [29]:
memreduced_all_df_feature.columns.values

array(['float面積mean_of_near_池袋駅', 'float面積std_of_near_池袋駅',
       'float面積min_of_near_池袋駅', ..., 'contract_periodmax_of_near_市川駅',
       '平均contract_period_diff_near_市川駅',
       '平均contract_period_percent_near_市川駅'], dtype=object)